In [1]:
import pickle
import pandas as pd
import numpy as np

## ml데이터 손보기

### ml데이터 제목 소문자, 연도 처리하기

In [2]:
ml_title_df = pd.read_csv('/opt/ml/input/fighting/CSV/ml-25m/movies.csv')

In [3]:
ml_title_df = ml_title_df[['movieId','title']]

In [4]:
ml_title_df.head(3)

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)


In [5]:
ml_title_df['title'] = ml_title_df['title'].str.lower()

In [6]:
ml_title_df.head(3)

,movieId,title
0,1,toy story (1995)
1,2,jumanji (1995)
2,3,grumpier old men (1995)


In [7]:
def del_year(title):
    try:
        idx = title.index("(")
        return title[:idx].strip()
    except:
        return title.strip()

In [8]:
ml_title_df['title'] = ml_title_df['title'].apply(del_year)

In [60]:
print(ml_title_df.shape)
ml_title_df.head(3)

(62423, 2)


,movieId,title
0,1,toy story
1,2,jumanji
2,3,grumpier old men


In [ ]:
# ml_data 중복제거 전 중복을 확인해봤다.
# 3229의 영화가 같은 이름을 가지고있었다.
# 이름이 같더라도 장르가 다른 완전 다른 영화인 것 같다.
# mbti에 있는 영화중 해당 이름을 가진 영화수는 477개
# 방법 1) 그냥 drop_duplicate 해버린다. -> 데이터 질은 낮을 수 있지만 빠름
# 방법 2) 제목 뿐만아니라 다른정보도 일일이 확인을 해서 매칭시켜준다. -> 시간은 오래걸리지만 데이터질은 높음
# 지금은 방법 1)을 채택

In [122]:
ml_title_df = ml_title_df.drop_duplicates(subset='title',keep="first")

In [124]:
ml_title_df

,movieId,title
0,1,toy story
1,2,jumanji
2,3,grumpier old men
3,4,waiting to exhale
4,5,father of the bride part ii
...,...,...
62417,209155,santosh subramaniam
62419,209159,window of the soul
62420,209163,bad poems
62421,209169,a girl thing


In [123]:
ml_title_df.nunique()

movieId    57893
title      57893
dtype: int64

## mbti - ml title 데이터_수작업x(교집합)

### mbti 소문자, 연도 처리

In [41]:
mbti_data = pd.read_csv('/opt/ml/input/fighting/CSV/Personality_All_168000_rows.csv')

In [42]:
category_dic = {
    'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_8.png' : 'anime & manga', # 애니 (예 : 하이큐 ...)
    'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_2.png' : 'television', # 드라마, 예능 
    'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_11.png' : 'n',
    'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_7.png' : 'cartoon', # 카툰 (티비 애니)
    'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_29.png' : 'n',
    'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_14.png' : 'n',
    'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_3.png' : 'movie', # 영화
    'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_1.png' : 'n',
    'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_27.png' : 'n', # <- 확인 필요
    'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_15.png' : 'n',
    'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_13.png' : 'n',
    'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_12.png' : 'n',
    'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_26.png' : 'web comic', # 웹툰 (예 : 네이버 웹툰, 카카오 웹툰 ...)
    'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_16.png' : 'n',
    'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_17.png' : 'n',
    'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_22.png' : 'n',
    'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_18.png' : 'n',
    'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_9.png' : 'comic', # 만화책
    'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_5.png' : 'n',
    'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_21.png' : 'n',
    'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_28.png' : 'n',
    'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_10.png' : 'n'
}

In [43]:
mbti_data['category'] = mbti_data['category'].map(category_dic)

In [44]:
mbti_data = mbti_data[mbti_data['category']=='movie']

In [50]:
mbti_data = mbti_data.drop_duplicates(subset='Contents', keep="first")

In [53]:
print(mbti_data.shape)
mbti_data.head(3)

(4291, 8)


,Character,Contents,Votes,MBTI,Enneagram,category,img_src,Page
14,Moana Waialiki,Moana (2016),2.7k/477,ENFJ,7w6,movie,https://static1.personality-database.com/profi...,1
16,Joy,Inside Out (2015),2.5k/358,ENFJ,7w6,movie,https://static1.personality-database.com/profi...,1
21,Judy Hopps,Zootopia (2016),2.3k/350,ENFJ,1w2,movie,https://static1.personality-database.com/profi...,1


In [54]:
mbti_data = mbti_data[['Contents']]

In [55]:
mbti_data.columns = ['title']

In [56]:
mbti_data['title'] = mbti_data['title'].str.lower()

/tmp/ipykernel_51400/2168468824.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mbti_data['title'] = mbti_data['title'].str.lower()


In [57]:
def del_year(title):
    try:
        idx = title.index("(")
        return title[:idx].strip()
    except:
        return title.strip()

In [58]:
mbti_data['title'] = mbti_data['title'].apply(del_year)

/tmp/ipykernel_51400/1205666247.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mbti_data['title'] = mbti_data['title'].apply(del_year)


In [59]:
mbti_data

,title
14,moana
16,inside out
21,zootopia
33,legally blonde
37,dead poets society
...,...
167846,dark waters
167855,let's play some mafia
167958,the young victoria
167972,a girl walks home alone at night


### mbti랑 ml 데이터 교집합 찾기

In [125]:
mbti_data.shape, ml_title_df.shape

((4291, 1), (57893, 2))

In [126]:
non_hand_intersection_mbti_ml = mbti_data.merge(ml_title_df, how='left', on='title')

In [127]:
non_hand_intersection_mbti_ml =  non_hand_intersection_mbti_ml.dropna()

In [128]:
non_hand_intersection_mbti_ml.drop_duplicates(subset=['title'], keep="first", inplace=True)

In [129]:
# 수작업을 거치지않은 mbti-ml 겹치는 제목 정보
non_hand_intersection_mbti_ml

,title,movieId
0,moana,73141.0
1,inside out,122470.0
2,zootopia,152081.0
3,legally blonde,4447.0
4,dead poets society,1246.0
...,...,...
4279,hot tub time machine,76077.0
4280,society,5606.0
4281,hide and seek,31427.0
4283,thor: ragnarok,122916.0


In [131]:
non_hand_intersection_mbti_ml.isna().sum()

title      0
movieId    0
dtype: int64

## mbti-ml title 데이터 _ 수작업결과(교집합)

### 시리즈 처리

In [134]:
import pickle

with open('/opt/ml/input/fighting/CSV/mbti_ml_match_dict.pkl', 'rb') as handle:
    match_dict = pickle.load(handle)

In [135]:
len(match_dict)

610

In [137]:
# value값이 리스트인 시리즈물 확인
for item in match_dict.items():
    if type(item[1]) == list:
        print(item)

In [8]:
def content_based_filtering_jaccard(title_list, title):
    topn=20
    target_split_set = set(title.split())
    sim_list = []
    sim_df = pd.DataFrame(title_list, columns=['title'])
    for idx, movie in enumerate(title_list):
        title_split_set = set(movie.split())
        title_intersection = target_split_set.intersection(title_split_set)
        jac_sim = float(len(title_intersection)) / (len(target_split_set) + len(title_split_set) - len(title_intersection))
        sim_list.append(jac_sim)
    sim_df['jaccard_similarity'] = sim_list

    return sim_df.sort_values('jaccard_similarity', ascending=False).reset_index(drop=True)[:topn]

In [ ]:
('the before trilogy', ['before sunrise', 'before sunset', 'before midnight'])
('the hangover trilogy', ['hangover part ii, the', 'hangover part iii, the'])
('chucky', ['bride of chucky', 'seed of chucky', 'cult of chucky'])
('sinister series', ['sinister', 'sinister 2'])
('three colours trilogy', ['three colors: red', 'three colors: blue', 'three colors: white'])
('ju-on', ['ju-on: the grudge', 'ju-on: the grudge 2', 'ju-on: white ghost', 'ju-on: the beginning of the end'])
('anchorman', ['anchorman: the legend of ron burgundy', 'anchorman 2: the legend continues'])
('journey to the center of the earth / journey 2: the mysterious island', ['journey to the center of the earth', 'journey 2: the mysterious island'])


In [78]:
title = 'journey 2: the mysterious island'
len(ml_df[ml_df['title']==title])

404

In [79]:
match_dict['journey to the center of the earth / journey 2: the mysterious island'] = 'journey to the center of the earth'

In [80]:
# value값이 리스트인 시리즈물은 이제 없다.
for item in match_dict.items():
    if type(item[1]) == list:
        print(item)

In [81]:
#피클 저장하기 pickle
with open('/opt/ml/input/fighting/CSV/mbti_ml_match_dict.pkl', 'wb') as f:
    pickle.dump(match_dict, f)

### 수작업 처리 결과 데이터(['title','movieId'])

In [138]:
with open('/opt/ml/input/fighting/CSV/mbti_ml_match_dict.pkl', 'rb') as handle:
    match_dict = pickle.load(handle)
ml_title = pd.read_csv("/opt/ml/input/fighting/CSV/ml_title.csv")
ml_title = ml_title.dropna(axis=0)

In [139]:
#{'mbti_title':'ml_title'} -> {'ml_title':'mbti_title'}
match_dict2 = {v:k for k, v in match_dict.items()}

In [140]:
ml_title['mbti_title'] = ml_title['title'].map(match_dict2)

In [141]:
ml_title = ml_title.dropna()

In [142]:
ml_title = ml_title.drop_duplicates(subset='title', keep='first')

In [143]:
ml_title

,title,mbti_title
31,twelve monkeys,12 monkeys / twelve monkeys
43,mortal kombat,mortal kombat films
49,"usual suspects, the",the usual suspects
72,"misérables, les",les miserables
103,"bridges of madison county, the",the bridges of madison county
...,...,...
60434,checkered ninja,ternet ninja
60626,turma da mônica: laços,turma da mônica
61007,"where'd you go, bernadette","where’d you go, bernadette"
61144,downton abbey,downtown abbey


In [144]:
# movieId정보 merge
ml_title = ml_title.merge(ml_title_df, how='left', on='title')

In [145]:
ml_title = ml_title.drop_duplicates(subset='title', keep='first')

In [146]:
ml_title

,title,mbti_title,movieId
0,twelve monkeys,12 monkeys / twelve monkeys,32
1,mortal kombat,mortal kombat films,44
2,"usual suspects, the",the usual suspects,50
3,"misérables, les",les miserables,73
4,"bridges of madison county, the",the bridges of madison county,105
...,...,...,...
593,checkered ninja,ternet ninja,203298
594,turma da mônica: laços,turma da mônica,203745
595,"where'd you go, bernadette","where’d you go, bernadette",204702
596,downton abbey,downtown abbey,205076


In [148]:
ml_title.isna().sum()

title         0
mbti_title    0
movieId       0
dtype: int64

In [149]:
# ml title 정보와 movieId 정보만 가져온다.
hand_intersection_mbti_ml = ml_title[['title', 'movieId']]

In [150]:
hand_intersection_mbti_ml

,title,movieId
0,twelve monkeys,32
1,mortal kombat,44
2,"usual suspects, the",50
3,"misérables, les",73
4,"bridges of madison county, the",105
...,...,...
593,checkered ninja,203298
594,turma da mônica: laços,203745
595,"where'd you go, bernadette",204702
596,downton abbey,205076


## mlti-ml 데이터 교집합 수작업/non 수작업 확인 및 합치기
-> 19개정도 겹친다.</br>
-> 하지만 결과는 동일하다.

### mlti-ml 데이터 교집합 수작업/non 수작업 확인

In [154]:
print(len(hand_intersection_mbti_ml))
print(len(non_hand_intersection_mbti_ml))

598
2650


In [155]:
set(list(hand_intersection_mbti_ml['title'])) & set(list(non_hand_intersection_mbti_ml['title']))

{'101 dalmatians',
 'barbie: a fairy secret',
 'bhool bhulaiyaa',
 "charlie's angels",
 'daisies',
 'dracula',
 'final destination',
 'flatliners',
 'lilya 4-ever',
 'mortal kombat',
 'my girl',
 'romeo and juliet',
 'scooby-doo and the ghoul school',
 'super mario bros.',
 'the game',
 'the hunger games',
 'the witch: part 1. the subversion',
 'up',
 'wrong turn'}

In [156]:
len(set(list(hand_intersection_mbti_ml['title'])) & set(list(non_hand_intersection_mbti_ml['title'])))

19

In [157]:
hand_intersection_mbti_ml[hand_intersection_mbti_ml['title']=='romeo and juliet']

,title,movieId
146,romeo and juliet,3668


In [158]:
# non_hand_intersection_mbti_ml[non_hand_intersection_mbti_ml['title']=='up']
non_hand_intersection_mbti_ml[non_hand_intersection_mbti_ml['title']=='romeo and juliet']

,title,movieId
1013,romeo and juliet,3668.0


### mbti-ml 교집합 데이터 (['title','movieId']) 합치기. 여기서의 title은 ml의 title이다.
전체 3248행. (총 mbti 데이터의 영화수 : 4291)</br>
수작업 안한 교집합 : 2650행</br>
수작업한 교집합 : 598행

In [159]:
intersection_mbti_ml_title_movieId_df = pd.concat([non_hand_intersection_mbti_ml, hand_intersection_mbti_ml])

In [160]:
intersection_mbti_ml_title_movieId_df['movieId'] = intersection_mbti_ml_title_movieId_df['movieId'].astype(int)

In [177]:
len(intersection_mbti_ml_title_movieId_df)

3248

In [185]:
# 수작업, no수작업 겹치는 19개에 대한 중복제거
intersection_mbti_ml_title_movieId_df = intersection_mbti_ml_title_movieId_df.drop_duplicates(subset=['title','movieId'], keep='first')

In [186]:
intersection_mbti_ml_title_movieId_df

,title,movieId
0,moana,73141
1,inside out,122470
2,zootopia,152081
3,legally blonde,4447
4,dead poets society,1246
...,...,...
593,checkered ninja,203298
594,turma da mônica: laços,203745
595,"where'd you go, bernadette",204702
596,downton abbey,205076


In [187]:
intersection_mbti_ml_title_movieId_df.to_csv('/opt/ml/input/fighting/CSV/inter_mbti_ml_title_movieId.csv',index=False)

In [188]:
!pwd

/opt/ml/input/fighting/EDA


## rating 정보에 교집합 해당하는 데이터만 추려내기

In [190]:
# ml_df는 원본 무비렌즈 interaction 데이터
ml_df = pd.read_csv('/opt/ml/input/fighting/CSV/ml-25m/ratings.csv')

In [191]:
ml_df

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


In [192]:
#rating 정보 merge
result_df = ml_df.merge(intersection_mbti_ml_title_movieId_df, how='left', on=['movieId'])

In [193]:
result_df

,userId,movieId,rating,timestamp,title
0,1,296,5.0,1147880044,pulp fiction
1,1,306,3.5,1147868817,three colors: red
2,1,307,5.0,1147868828,NaN
3,1,665,5.0,1147878820,NaN
4,1,899,3.5,1147868510,singin' in the rain
...,...,...,...,...,...
25000090,162541,50872,4.5,1240953372,ratatouille
25000091,162541,55768,2.5,1240951998,bee movie
25000092,162541,56176,2.0,1240950697,alvin and the chipmunks
25000093,162541,58559,4.0,1240953434,NaN


In [194]:
result_df.isna().sum()

userId              0
movieId             0
rating              0
timestamp           0
title        12145460
dtype: int64

In [195]:
result_df.dropna()

,userId,movieId,rating,timestamp,title
0,1,296,5.0,1147880044,pulp fiction
1,1,306,3.5,1147868817,three colors: red
4,1,899,3.5,1147868510,singin' in the rain
5,1,1088,4.0,1147868495,dirty dancing
7,1,1217,3.5,1147878326,ran
...,...,...,...,...,...
25000089,162541,45517,4.5,1240953353,cars
25000090,162541,50872,4.5,1240953372,ratatouille
25000091,162541,55768,2.5,1240951998,bee movie
25000092,162541,56176,2.0,1240950697,alvin and the chipmunks


In [196]:
result_df.to_csv('/opt/ml/input/fighting/CSV/annoy_input_ver1.csv', index=False)

In [197]:
result_df.columns

Index(['userId', 'movieId', 'rating', 'timestamp', 'title'], dtype='object')

In [198]:
result_df.nunique()

userId         162541
movieId         59047
rating             10
timestamp    20115267
title            3213
dtype: int64

### 결과파일 확인하기 - 왜 겹치는 영화수는 3229였는데 결과파일의 고유영화수는 3213일까?
-> 영화정보는 있으나, interaction이 전혀 없는 영화이기 때문이다.</br>
아래 코드를 보면, interaction를 한번이라도 한 영화는 58047개이지만, 영화 title 수는 62423개이다.

In [199]:
rating_df = pd.read_csv('/opt/ml/input/fighting/CSV/ml-25m/ratings.csv')
title_df = pd.read_csv('/opt/ml/input/fighting/CSV/ml-25m/movies.csv')

In [200]:
rating_df.drop_duplicates(subset='movieId', keep='first')

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
24966301,162358,200192,2.0,1553453039
24966302,162358,200194,2.0,1553453843
24970551,162386,139970,3.5,1549215965
24970609,162386,200726,4.0,1554651417


In [201]:
title_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)
